In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import pymongo

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw
from climatools.cliradlw.utils import mapband_new2old

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *

import IPython.display as display

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
def make_query(param=None):
    return {'param.' + name: value for name, value in param.items()}   

In [4]:
def show_html(s): display.display(display.HTML(s))
    
def show_markdown(s): display.display(display.Markdown(s))

In [5]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [6]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)

    for dim in ['band', 'g']: 
        if dim in ds: ds = ds.sum(dim)

    for dim in ['igg', 'g', 'i']:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim) 
    return ds['coolrg']

def pltdata_cooling_1():
    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(DS_COOL.sel(i=1)),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(DS_COOL_CRD),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(DS_COOL_WGT.sel(igg=10)),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data

def nice_xlims(pltdata=None, prange=None):
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)

def plt_cool(pltdata=None):
    plin = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='linear',
                                  prange=(50, 1050))
    plin.plot_width, plin.plot_height = 400, 500
    plog = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='log',
                                  prange=(.01, 200))
    plog.plot_width, plog.plot_height = 400, 500
    everything = gridplot(plin, plog, ncols=2)
    return everything
    
def script_plt_cooling():
    pltdata = pltdata_cooling_1()
    p = plt_cool(pltdata=pltdata)
    show(p)
    show_markdown('''*FIGURE.* Cooling rate profiles.''')

In [7]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels: ds = ds.isel(pressure=ilevels)

    for dim in ['band', 'g']:
        if dim in ds: ds = ds.sum(dim)
        
    for dim in ['i', 'igg', 'g']:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df

def tbdata_flux():
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    atmpro = PARAM['atmpro']
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(DS_FLUX_CRD, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10',
            'df': fmt_flux(DS_FLUX_WGT.sel(igg=10), ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(DS_FLUX.sel(i=1), ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data

def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        show_markdown(f'''*TABLE.*  Fluxes. {d['label']}''')
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        show_markdown(
            f'''*TABLE.*  Fluxes.  ({d['label']}) - ({d0['label']})''')
        
    show_tb(benchmark)
    for d in others: show_tb(d)
    for d in others: show_tbdiff(d=d, d0=benchmark)
        
def script_tb_flux():
    data = tbdata_flux()
    tb_flux(tbdata=data)

In [8]:
def gasbands():
    bmap = mapband_new2old()
    gases = [gas for gas, _ in PARAM['molecule'].items()] 
    bands = [bmap[b] for b in PARAM['band']]
    return list(itertools.product(gases, bands))

def print_bestfit_params():
    df = pd.DataFrame()
    for gas, band in gasbands():
        param = bestfits.kdist_params(molecule=gas, band=band)
        srs = pd.Series(param)
        df[f'({gas}, {band})'] = srs
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df.sort_index())
    pd.reset_option('display.max_colwidth')
        
def print_input_param():
    df_clirad = pd.Series(PARAM, name='clirad run').to_frame()
    df_lblnew = pd.Series(PARAM_LBLNEW, name='lblnew run compared against').to_frame()
    df = pd.merge(df_clirad, df_lblnew, how='outer', 
                  left_index=True, right_index=True)
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df)
    pd.reset_option('display.max_colwidth')
        
def print_lblnew_param():
    pd.set_option('display.max_colwidth', -1)
    display.display(pd.Series(PARAM_LBLNEW).to_frame())
    pd.reset_option('display.max_colwidth')

In [9]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL.sel(i=1)]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [10]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX.sel(i=1)]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [11]:
def script_fluxcoolr_excelfile():
    '''
    Save fluxes and cooling rates by layer by g-interval in excel file
    and make it available for download.
    '''
    fname = 'output_byg.xlsx'
    
    writer = pd.ExcelWriter(fname)
    
    for g in DS_FLUX_WGT.coords['g']:
        df = DS_FLUX_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'flux g={int(g)}')
        
    for g in DS_COOL_WGT.coords['g']:
        df = DS_COOL_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'cool g={int(g)}')    

In [12]:
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()

In [13]:
%run param.py

client = pymongo.MongoClient('localhost', 27017)

if 'ng_refs' in PARAM_LBLNEW: 
    doc = client.lblnew.bestfit_lw.find_one(make_query(param=PARAM_LBLNEW))
    DS_FLUX_CRD = load_output_file(io.StringIO(doc['output_fluxg']))
    DS_COOL_CRD = load_output_file(io.StringIO(doc['output_coolrg']))
    DS_FLUX_WGT = load_output_file(io.StringIO(doc['output_wfluxg']))
    DS_COOL_WGT = load_output_file(io.StringIO(doc['output_wcoolrg']))
else:
    doc = client.lblnew.overlap_lw.find_one(make_query(param=PARAM_LBLNEW))
    DS_FLUX_CRD = load_output_file(io.StringIO(doc['output_flux']))
    DS_COOL_CRD = load_output_file(io.StringIO(doc['output_coolr']))
    DS_FLUX_WGT = load_output_file(io.StringIO(doc['output_wflux']))
    DS_COOL_WGT = load_output_file(io.StringIO(doc['output_wcoolr']))

doc = client.cliradnew.lw.find_one(make_query(param=PARAM))
DS_FLUX = load_output_file(io.StringIO(doc['output_flux']))
DS_COOL = load_output_file(io.StringIO(doc['output_coolr']))

In [14]:
script()
if 'ng_refs' in PARAM_LBLNEW:
    script_fluxcoolr_excelfile()

### CLIRAD-LW

### Table of Contents

# Input Parameters

,clirad run,lblnew run compared against
atmpro,trp,trp
band,[6],4
commitnumber,1013f91,5014a19
dv,-,0.001
molecule,"{'co2': 0.0004, 'h2o': 'atmpro'}","{'h2o': 'atmpro', 'co2': 0.0004}"
nv,-,1000
tsfc,300,300


# Best-fit Parameters

...

*TABLE.*  Fluxes.  (CLIRAD) - (CRD)

# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312        1 -0.000820                    1 -0.000890
0.000750        2 -0.000823                    2 -0.000917
0.001052        3 -0.000829                    3 -0.000892
0.001476        4 -0.000835                    4 -0.000901
0.002070        5 -0.000839                    5 -0.000908
0.002904        6 -0.000844                    6 -0.000916
0.004074        7 -0.000847                    7 -0.000922
0.005714        8 -0.000850                    8 -0.000928
0.008015        9 -0.000852                    9 -0.000934
0.011243       10 -0.000854                   10 -0.000941
0.015771       11 -0.000855                   11 -0.000953
0.022122       12 -0.000855                   12 -0.000962
0.031031       13 -0.000853                   13 -0.000971
0.043528       14 -0.000851                   14 -0.000982
0.061057       15 -0.000827                   15 -0.000999
0.085645       16 -0.000695                   16 -0.000975
0.120136       17 -0.000334                   17 -0.000820
0.168516       18  0.000367                   18 -0.000450
0.236378       19  0.001576                   19  0.000239
0.331549       20  0.003521                   20  0.001392
0.465100       21  0.006499                   21  0.003241
0.652400       22  0.010388                   22  0.005875
0.915100       23  0.011825                   23  0.007009
1.283650       24  0.009931                   24  0.005926
1.800600       25  0.007210                   25  0.004247
2.525700       26  0.004391                   26  0.002454
3.542800       27  0.002243                   27  0.001061
4.969550       28  0.000770                   28  0.000092
6.970850       29 -0.000198                   29 -0.000560
9.778100       30 -0.000802                   30 -0.000975
13.715850      31 -0.001145                   31 -0.001213
19.239350      32 -0.001310                   32 -0.001321
26.987250      33 -0.001355                   33 -0.001328
37.855300      34 -0.001320                   34 -0.001248
53.100050      35 -0.001187                   35 -0.001132
73.887500      36 -0.000967                   36 -0.000983
97.662500      37 -0.000840                   37 -0.000885
121.437500     38 -0.000960                   38 -0.000995
145.212500     39 -0.001226                   39 -0.001210
168.987500     40 -0.001464                   40 -0.001390
192.762500     41 -0.001661                   41 -0.001574
216.537500     42 -0.001814                   42 -0.001779
240.312500     43 -0.001898                   43 -0.001924
264.087500     44 -0.001880                   44 -0.001984
287.862500     45 -0.001700                   45 -0.001937
311.637500     46 -0.001268                   46 -0.001728
335.412500     47 -0.000461                   47 -0.001241
359.187500     48  0.000845                   48 -0.000381
382.962500     49  0.002607                   49  0.000921
406.737500     50  0.005004                   50  0.002816
430.512500     51  0.008130                   51  0.005411
454.287500     52  0.012062                   52  0.008866
478.062500     53  0.016865                   53  0.013330
501.837500     54  0.022557                   54  0.018862
525.612500     55  0.029198                   55  0.025337
549.387500     56  0.036779                   56  0.032879
573.162500     57  0.045321                   57  0.041480
596.937500     58  0.054881                   58  0.051075
620.712500     59  0.065443                   59  0.061558
644.487500     60  0.077082                   60  0.072705
668.262500     61  0.089670                   61  0.084472
692.037500     62  0.102722                   62  0.096767
715.812500     63  0.115774                   63  0.109430
739.587500     64  0.135698                   64  0.129476
763.362500     65  0.153220                   65  0.148649
78

# Fluxes by Layer

CRD                              CLIRAD  \
                        flug          fldg      fnetg       flug   
pressure    level                                                  
0.000000    1     -64.984003  0.000000e+00 -64.984003 -65.022532   
0.000624    2     -64.984003  3.280962e-08 -64.984003 -65.022532   
0.000876    3     -64.984003  4.630052e-08 -64.984003 -65.022533   
0.001229    4     -64.984003  6.602648e-08 -64.984003 -65.022533   
0.001723    5     -64.984003  9.489255e-08 -64.984003 -65.022533   
0.002417    6     -64.984003  1.371194e-07 -64.984003 -65.022533   
0.003391    7     -64.984004  1.988890e-07 -64.984003 -65.022533   
0.004757    8     -64.984004  2.892351e-07 -64.984004 -65.022533   
0.006672    9     -64.984004  4.213527e-07 -64.984004 -65.022533   
0.009359    10    -64.984005  6.145316e-07 -64.984004 -65.022534   
0.013128    11    -64.984005  8.969494e-07 -64.984004 -65.022535   
0.018415    12    -64.984006  1.309780e-06 -64.984005 -65.022535   
0.025830    13    -64.984008  1.913161e-06 -64.984006 -65.022537   
0.036232    14    -64.984010  2.795062e-06 -64.984007 -65.022539   
0.050823    15    -64.984012  4.098441e-06 -64.984008 -65.022541   
0.071291    16    -64.984017  6.290120e-06 -64.984010 -65.022545   
0.100000    17    -64.984023  1.066061e-05 -64.984013 -65.022552   
0.140271    18    -64.984034  1.979957e-05 -64.984014 -65.022563   
0.196760    19    -64.984050  3.869123e-05 -64.984012 -65.022579   
0.275997    20    -64.984074  7.727446e-05 -64.983997 -65.022605   
0.387100    21    -64.984106  1.549986e-04 -64.983951 -65.022641   
0.543100    22    -64.984140  3.095675e-04 -64.983831 -65.022690   
0.761700    23    -64.984164  6.020336e-04 -64.983562 -65.022751   
1.068500    24    -64.984179  1.047015e-03 -64.983132 -65.022831   
1.498800    25    -64.984220  1.594226e-03 -64.982626 -65.022955   
2.102400    26    -64.984320  2.209576e-03 -64.982110 -65.023150   
2.949000    27    -64.984518  2.847857e-03 -64.981670 -65.023445   
4.136600    28    -64.984847  3.493180e-03 -64.981354 -65.023869   
5.802500    29    -64.985346  4.143561e-03 -64.981202 -65.024453   
8.139200    30    -64.986057  4.799947e-03 -64.981257 -65.025237   
11.417000   31    -64.987034  5.466083e-03 -64.981568 -65.026267   
16.014700   32    -64.988339  6.147216e-03 -64.982192 -65.027597   
22.464000   33    -64.990042  6.848692e-03 -64.983193 -65.029291   
31.510500   34    -64.992218  7.573494e-03 -64.984645 -65.031405   
44.200100   35    -64.994943  8.313998e-03 -64.986629 -65.033961   
62.000000   36    -64.998102  8.970051e-03 -64.989132 -65.036961   
85.775000   37    -65.001303  9.447689e-03 -64.991855 -65.040219   
109.550000  38    -65.004006  9.785843e-03 -64.994221 -65.043073   
133.325000  39    -65.007181  1.025596e-02 -64.996925 -65.046348   
157.100000  40    -65.011508  1.112835e-02 -65.000379 -65.050561   
180.875000  41    -65.017062  1.255884e-02 -65.004503 -65.055717   
204.650000  42    -65.023907  1.472586e-02 -65.009181 -65.061994   
228.425000  43    -65.032183  1.789335e-02 -65.014290 -65.069729   
252.200000  44    -65.042053  2.241809e-02 -65.019635 -65.079046   
275.975000  45    -65.053768  2.883849e-02 -65.024929 -65.090093   
299.750000  46    -65.067683  3.796460e-02 -65.029718 -65.103175   
323.525000  47    -65.084266  5.097551e-02 -65.033291 -65.118770   
347.300000  48    -65.104095  6.950653e-02 -65.034589 -65.137270   
371.075000  49    -65.127726  9.551687e-02 -65.032209 -65.159312   
394.850000  50    -65.155067  1.302005e-01 -65.024866 -65.184970   
418.625000  51    -65.186514  1.757419e-01 -65.010772 -65.214595   
442.400000  52    -65.222459  2.345818e-01 -64.987877 -65.248442   
466.175000  53    -65.263270  3.093630e-01 -64.953907 -65.287042   
489.950000  54    -65.309289  4.028805e-01 -64.906408 -65.330870   
513.725000  55    -65.360753  5.178720e-01 -64.842881 -65.380294   
537.500000  56    -65.417888  6.572375e-01 -64.760650 -65.435146   
561.275000  5

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')